Data Preproccesing

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('result.csv')
print(df['rating'].value_counts())

rating
5.0    34180
4.0     7616
3.0     3404
1.0     2097
2.0     1095
Name: count, dtype: int64


TAHAP EKSTRAKSI FITUR DAN PELABELAN DATA

In [3]:
def clean_comment(teks):
    if pd.isna(teks):
        return ""

    teks = str(teks).lower()

    # emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    teks = emoji_pattern.sub(r'', teks)

    # URL
    teks = re.sub(r'http\S+|www\S+|https\S+', '', teks)

    # mention dan hashtag
    teks = re.sub(r'@\w+|#\w+', '', teks)

    # karakter khusus
    teks = re.sub(r'[^a-z\s]', ' ', teks)

    # spasi berlebih
    teks = re.sub(r'\s+', ' ', teks).strip()

    return teks

df['comment_clean'] = df['comment'].apply(clean_comment)

# empty comment
df = df[df['comment_clean'].str.len() > 0]

# duplikat
print(f"Duplikat ditemukan: {df.duplicated(subset=['comment_clean']).sum()}")
df = df.drop_duplicates(subset=['comment_clean'], keep='first')
print(f"Data setelah hapus duplikat: {df.shape}")


Duplikat ditemukan: 2746
Data setelah hapus duplikat: (45510, 3)


In [4]:
def label_sentimen(rating):
    if rating >= 4.0:
        return 2
    elif rating >= 3.0:
        return 1
    else:
        return 0


df['sentimen'] = df['rating'].apply(label_sentimen)

print("Distribusi sentimen:")
print(df['sentimen'].value_counts())
print("Persentase:")
print(df['sentimen'].value_counts(normalize=True) * 100)

Distribusi sentimen:
sentimen
2    39071
1     3286
0     3153
Name: count, dtype: int64
Persentase:
sentimen
2    85.851461
1     7.220391
0     6.928148
Name: proportion, dtype: float64


In [5]:
MAX_WORDS = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(df['comment_clean'])

sequences = tokenizer.texts_to_sequences(df['comment_clean'])
X = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')
y = df['sentimen'].values

print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")

Shape X: (45510, 100)
Shape y: (45510,)


# **PERCOBAAN 1**

Ekstraksi Fitur : Word embedding

Model : LSTM

Train/val/test : 72/8/20

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

Train: (32767, 100), Val: (3641, 100), Test: (9102, 100)


In [7]:
# Karena dataset tidak imbang tambahakan weight
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = {i: weight for i, weight in enumerate(class_weights_array)}
print(f"Class weights: {class_weights}")

Class weights: {0: np.float64(4.811600587371513), 1: np.float64(4.616370808678501), 2: np.float64(0.3882667993790954)}


TAHAP PELATIHAN MODEL

In [8]:
model_lstm = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  #  negatif, netral, positif
])

model_lstm.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_lstm.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=0.00001,
    verbose=1
)

In [10]:
history = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/30
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 31s 21ms/step - accuracy: 0.6520 - loss: 0.8838 - val_accuracy: 0.8624 - val_loss: 0.4684 - learning_rate: 0.0010
Epoch 2/30
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.8422 - loss: 0.6030 - val_accuracy: 0.8099 - val_loss: 0.5427 - learning_rate: 0.0010
Epoch 3/30
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.8746 - loss: 0.4754 - val_accuracy: 0.8610 - val_loss: 0.4422 - learning_rate: 0.0010
Epoch 4/30
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.8895 - loss: 0.4095 - val_accuracy: 0.8121 - val_loss: 0.5559 - learning_rate: 0.0010
Epoch 5/30
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.9045 - loss: 0.3361 - val_accuracy: 0.8168 - val_loss: 0.5557 - learning_rate: 0.0010
Epoch 6/30
1022/1024 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9141 - loss: 0.2919
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy

TAHAP EVALUASI

In [12]:
# evaluasi pada test set
test_loss, test_acc = model_lstm.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

y_pred = model_lstm.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(
    y_test,
    y_pred_classes,
    target_names=['Negatif', 'Netral', 'Positif']
))


Test Accuracy: 0.8688
Test Loss: 0.4311
285/285 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Classification Report:
              precision    recall  f1-score   support

     Negatif       0.64      0.72      0.68       631
      Netral       0.30      0.45      0.36       657
     Positif       0.97      0.92      0.94      7814

    accuracy                           0.87      9102
   macro avg       0.63      0.70      0.66      9102
weighted avg       0.90      0.87      0.88      9102



In [15]:
def prediksi_sentimen(teks):
    cleaned_comment = clean_comment(teks)
    seq = tokenizer.texts_to_sequences([cleaned_comment])
    padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    pred = model_lstm.predict(padded, verbose=0)
    kelas = np.argmax(pred, axis=1)[0]
    confidence = np.max(pred) * 100

    label = ['Negatif', 'Netral', 'Positif'][kelas]
    return label, confidence, pred[0]

# test prediksi
contoh_komentar = [
    "Tokonya bagus banget, pelayanan ramah, barang cepat sampai!",
    "Biasa aja sih, tidak terlalu istimewa",
    "Mengecewakan sekali, barang tidak sesuai dan lama banget",
    "Pengiriman cepat tapi kualitas produk standar",
    "Sangat puas dengan pembelian ini, recommended!"
]

for komentar in contoh_komentar:
    label, conf, prob = prediksi_sentimen(komentar)
    print(f"Komentar: {komentar}")
    print(f"Prediksi: {label} ({conf:.2f}%)")
    print(f"Probabilitas -> Negatif: {prob[0]:.2%}, Netral: {prob[1]:.2%}, Positif: {prob[2]:.2%}")

Komentar: Tokonya bagus banget, pelayanan ramah, barang cepat sampai!
Prediksi: Positif (85.08%)
Probabilitas -> Negatif: 0.02%, Netral: 14.90%, Positif: 85.08%
Komentar: Biasa aja sih, tidak terlalu istimewa
Prediksi: Netral (54.15%)
Probabilitas -> Negatif: 44.49%, Netral: 54.15%, Positif: 1.36%
Komentar: Mengecewakan sekali, barang tidak sesuai dan lama banget
Prediksi: Negatif (98.32%)
Probabilitas -> Negatif: 98.32%, Netral: 1.66%, Positif: 0.02%
Komentar: Pengiriman cepat tapi kualitas produk standar
Prediksi: Netral (88.63%)
Probabilitas -> Negatif: 8.09%, Netral: 88.63%, Positif: 3.28%
Komentar: Sangat puas dengan pembelian ini, recommended!
Prediksi: Positif (96.79%)
Probabilitas -> Negatif: 0.00%, Netral: 3.21%, Positif: 96.79%




---


# **PERCOBAAN 2**

Ekstraksi fitur : TF-IDF

Model : SVM

train/val/test : 72/8/20

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import pickle
from collections import Counter


In [17]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 3),  # unigram bigram trigram
    min_df=2,  # minimal muncul di 2 dokumen
    max_df=0.8,  # maksimal muncul di 80% dokumen
    sublinear_tf=True  # scaling frekuensi
)

In [35]:
X = df['comment_clean'].values
y = df['sentimen'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

# simpan untuk simpulan
y_test_svm = y_test

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

Train: 32767, Val: 3641, Test: 9102


In [19]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

print(f"Shape X_train_tfidf: {X_train_tfidf.shape}")
print(f"Shape X_val_tfidf: {X_val_tfidf.shape}")
print(f"Shape X_test_tfidf: {X_test_tfidf.shape}")


Shape X_train_tfidf: (32767, 5000)
Shape X_val_tfidf: (3641, 5000)
Shape X_test_tfidf: (9102, 5000)


In [20]:
svm_model = SVC(
    kernel='rbf',
    C=10,
    gamma='scale',
    class_weight=class_weights,
    random_state=42,
    probability=True,
    verbose=True
)

print("training...")
svm_model.fit(X_train_tfidf, y_train)
print("Training selesai!")


training...
[LibSVM]Training selesai!


In [21]:
y_pred_svm = svm_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred_svm)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred_svm))

Accuracy: 0.9033179520984399
              precision    recall  f1-score   support

           0       0.69      0.67      0.68       631
           1       0.41      0.20      0.26       657
           2       0.94      0.98      0.96      7814

    accuracy                           0.90      9102
   macro avg       0.68      0.62      0.64      9102
weighted avg       0.88      0.90      0.89      9102



In [22]:
def prediksi_sentimen(teks):
    teks_bersih = clean_comment(teks)
    teks_tfidf = tfidf.transform([teks_bersih])
    pred = svm_model.predict(teks_tfidf)[0]
    prob = svm_model.predict_proba(teks_tfidf)[0]
    confidence = max(prob) * 100

    label = ['Negatif', 'Netral', 'Positif'][pred]
    return label, confidence, prob

contoh_komentar = [
    "Tokonya bagus banget, pelayanan ramah, barang cepat sampai!",
    "Biasa aja sih, tidak terlalu istimewa",
    "Mengecewakan sekali, barang tidak sesuai dan lama banget",
    "Pengiriman cepat tapi kualitas produk standar",
    "Sangat puas dengan pembelian ini, recommended!"
]

for komentar in contoh_komentar:
    label, conf, prob = prediksi_sentimen(komentar)
    print(f"Komentar: {komentar}")
    print(f"Prediksi: {label} ({conf:.2f}%)")
    print(f"Probabilitas -> Negatif: {prob[0]:.2%}, Netral: {prob[1]:.2%}, Positif: {prob[2]:.2%}")


Komentar: Tokonya bagus banget, pelayanan ramah, barang cepat sampai!
Prediksi: Positif (98.28%)
Probabilitas -> Negatif: 0.05%, Netral: 1.67%, Positif: 98.28%
Komentar: Biasa aja sih, tidak terlalu istimewa
Prediksi: Netral (66.77%)
Probabilitas -> Negatif: 14.88%, Netral: 66.77%, Positif: 18.36%
Komentar: Mengecewakan sekali, barang tidak sesuai dan lama banget
Prediksi: Negatif (96.84%)
Probabilitas -> Negatif: 96.84%, Netral: 2.51%, Positif: 0.65%
Komentar: Pengiriman cepat tapi kualitas produk standar
Prediksi: Positif (70.32%)
Probabilitas -> Negatif: 3.01%, Netral: 26.67%, Positif: 70.32%
Komentar: Sangat puas dengan pembelian ini, recommended!
Prediksi: Positif (98.41%)
Probabilitas -> Negatif: 0.26%, Netral: 1.33%, Positif: 98.41%


# **PERCOBAAN 3**
Ekstraksi fitur : Word Embedding

Model : GRU

train/val/test : 70/15/15

In [23]:
from tensorflow.keras.layers import SimpleRNN, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [24]:

MAX_WORDS = 10000
MAX_LEN = 100
EMBEDDING_DIM = 128

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(df['comment_clean'])

sequences = tokenizer.texts_to_sequences(df['comment_clean'])
X = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')
y = df['sentimen'].values

print(f"Vocabulary size: {len(tokenizer.word_index)}")
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")

Vocabulary size: 20559
Shape X: (45510, 100)
Shape y: (45510,)


In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42,
    stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.176,
    random_state=42,
    stratify=y_train
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

Train: (31874,), Val: (6809,), Test: (6827,)


In [26]:
model_gru = Sequential([
  Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
  Bidirectional(GRU(128, return_sequences=True)),
  Dropout(0.5),
  Bidirectional(GRU(64)),
  Dropout(0.5),
  Dense(64, activation='relu'),
  Dropout(0.3),
  Dense(3, activation='softmax')
])

model_gru.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_gru.summary()

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=0.00001,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_rnn_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model_gru.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)

Epoch 1/30
995/997 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7333 - loss: 0.8477
Epoch 1: val_accuracy improved from -inf to 0.65311, saving model to best_rnn_model.h5


997/997 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.7335 - loss: 0.8475 - val_accuracy: 0.6531 - val_loss: 0.7420 - learning_rate: 0.0010
Epoch 2/30
997/997 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8400 - loss: 0.5887
Epoch 2: val_accuracy improved from 0.65311 to 0.86591, saving model to best_rnn_model.h5


997/997 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.8400 - loss: 0.5887 - val_accuracy: 0.8659 - val_loss: 0.4113 - learning_rate: 0.0010
Epoch 3/30
997/997 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8738 - loss: 0.4709
Epoch 3: val_accuracy did not improve from 0.86591
997/997 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.8738 - loss: 0.4709 - val_accuracy: 0.8611 - val_loss: 0.4258 - learning_rate: 0.0010
Epoch 4/30
996/997 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8904 - loss: 0.4061
Epoch 4: val_accuracy improved from 0.86591 to 0.86665, saving model to best_rnn_model.h5


997/997 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.8903 - loss: 0.4061 - val_accuracy: 0.8666 - val_loss: 0.4456 - learning_rate: 0.0010
Epoch 5/30
995/997 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9021 - loss: 0.3210
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 5: val_accuracy did not improve from 0.86665
997/997 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.9021 - loss: 0.3211 - val_accuracy: 0.8431 - val_loss: 0.5238 - learning_rate: 0.0010
Epoch 6/30
995/997 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9243 - loss: 0.2460
Epoch 6: val_accuracy did not improve from 0.86665
997/997 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.9243 - loss: 0.2460 - val_accuracy: 0.8458 - val_loss: 0.5610 - learning_rate: 5.0000e-04
Epoch 7/30
995/997 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9268 - loss: 0.2093
Epoch 7: val_accuracy did not improve from 0.86665
997/997 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.9268 - loss: 0.2093

In [29]:
test_loss, test_acc = model_gru.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

y_pred = model_gru.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

print("Classification Report:")
print(classification_report(
    y_test,
    y_pred_classes,
    target_names=['Negatif', 'Netral', 'Positif'],
    digits=4
))

for i, label in enumerate(['Negatif', 'Netral', 'Positif']):
    mask = y_test == i
    if mask.sum() > 0:
        class_acc = (y_pred_classes[mask] == i).sum() / mask.sum()
        print(f"Akurasi {label}: {class_acc:.4f}")

Test Accuracy: 0.8686
Test Loss: 0.3920
Classification Report:
              precision    recall  f1-score   support

     Negatif     0.6275    0.6660    0.6462       473
      Netral     0.3168    0.5172    0.3929       493
     Positif     0.9710    0.9145    0.9419      5861

    accuracy                         0.8686      6827
   macro avg     0.6384    0.6992    0.6603      6827
weighted avg     0.9000    0.8686    0.8818      6827

Akurasi Negatif: 0.6660
Akurasi Netral: 0.5172
Akurasi Positif: 0.9145


In [31]:
embedding_layer = model_gru.layers[0]
embeddings = embedding_layer.get_weights()[0]

print(f"Shape embedding matrix: {embeddings.shape}")
print(f"Jumlah kata dalam vocabulary: {embeddings.shape[0]}")
print(f"Dimensi embedding per kata: {embeddings.shape[1]}")

Shape embedding matrix: (10000, 128)
Jumlah kata dalam vocabulary: 10000
Dimensi embedding per kata: 128


In [32]:
def prediksi_sentimen(teks):
    teks_bersih = clean_comment(teks)
    seq = tokenizer.texts_to_sequences([teks_bersih])
    padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    pred = model_gru.predict(padded, verbose=0)
    kelas = np.argmax(pred, axis=1)[0]
    confidence = np.max(pred) * 100
    prob = pred[0]

    label = ['Negatif', 'Netral', 'Positif'][kelas]
    return label, confidence, prob

contoh_komentar = [
    "Tokonya bagus banget, pelayanan ramah, barang cepat sampai!",
    "Biasa aja sih, tidak terlalu istimewa",
    "Mengecewakan sekali, barang tidak sesuai dan lama banget",
    "Pengiriman cepat tapi kualitas produk standar",
    "Sangat puas dengan pembelian ini, recommended!",
    "Buruk, tidak akan beli lagi di sini"
]

for komentar in contoh_komentar:
    label, conf, prob = prediksi_sentimen(komentar)
    print(f"Komentar: {komentar}")
    print(f"Prediksi: {label} ({conf:.2f}%)")
    print(f"Probabilitas -> Negatif: {prob[0]:.2%}, Netral: {prob[1]:.2%}, Positif: {prob[2]:.2%}")

Komentar: Tokonya bagus banget, pelayanan ramah, barang cepat sampai!
Prediksi: Positif (94.60%)
Probabilitas -> Negatif: 0.03%, Netral: 5.36%, Positif: 94.60%
Komentar: Biasa aja sih, tidak terlalu istimewa
Prediksi: Netral (64.67%)
Probabilitas -> Negatif: 33.29%, Netral: 64.67%, Positif: 2.05%
Komentar: Mengecewakan sekali, barang tidak sesuai dan lama banget
Prediksi: Negatif (94.09%)
Probabilitas -> Negatif: 94.09%, Netral: 5.89%, Positif: 0.02%
Komentar: Pengiriman cepat tapi kualitas produk standar
Prediksi: Netral (58.92%)
Probabilitas -> Negatif: 11.55%, Netral: 58.92%, Positif: 29.52%
Komentar: Sangat puas dengan pembelian ini, recommended!
Prediksi: Positif (96.50%)
Probabilitas -> Negatif: 0.01%, Netral: 3.49%, Positif: 96.50%
Komentar: Buruk, tidak akan beli lagi di sini
Prediksi: Negatif (84.93%)
Probabilitas -> Negatif: 84.93%, Netral: 14.96%, Positif: 0.11%


# **INTERPRETASI DAN KESIMPULAN**


## LSTM
Accuracy : 0.8688

## SVM
Accuracy : 0.9033

## GRU
Accuracy : 0.8686

**### Penjelasan**

Perbedaan performa antar percobaan terutama dipengaruhi oleh metode ekstraksi fitur, jenis model, serta kondisi data yang tidak seimbang. Pada Percobaan 2 (TF-IDF + SVM) mempunyai akurasi yang tertinggi karena SVM sangat efektif memisahkan kelas dominan menggunakan fitur kata eksplisit, sehingga model cenderung memprediksi kelas positif yang jumlahnya sangat besar. Namun, pendekatan ini mengorbankan kemampuan mengenali kelas minoritas, terutama kelas netral yang bisa dilihat dari nilai recall netral yang sangat rendah.

Pada Percobaan 1 (Word Embedding + LSTM), model mampu menangkap konteks dan urutan kata sehingga lebih sensitif terhadap sentimen negatif, tetapi kompleksitas LSTM membuatnya kurang optimal pada data yang imbalanced dan menyebabkan kesulitan dalam membedakan kelas netral yang secara linguistik ambigu. Sementara itu, Percobaan 3 (Word Embedding + GRU) menunjukkan performa yang lebih seimbang karena arsitektur GRU yang lebih sederhana dan stabil mampu mengurangi overfitting terhadap kelas mayoritas. Hal ini terlihat dari nilai loss yang lebih rendah serta recall kelas netral yang paling tinggi, sehingga meskipun akurasi total tidak setinggi SVM, GRU memberikan representasi performa yang lebih adil untuk seluruh kelas sentimen.